In [ ]:
#Dependencies
! pip install requests        # To get HTML file
! pip install BeautifulSoup4  # For HTML parsing
import os
import sys
import time
import math
import requests
import pandas as pd
from google.colab import drive

In [ ]:
PAGE_MAX = 300
POP_URL_BASE = "https://www.psacard.com/Pop/GetSetItems"
EXAMPLE_URL = "https://www.psacard.com/pop/baseball-cards/1952/topps/49722"

SET_ID = "49722"

def scrape(pop_url, set_name):
    # Get json data for input set
    sess = requests.Session()
    sess.mount("https://", requests.adapters.HTTPAdapter(max_retries=5))
    form_data = {
        "headingID": SET_ID,
        "categoryID": "20019",
        "draw": 1,
        "start": 0,
        "length": 500,
        "isPSADNA": "false"
    }

    json_data = post_to_url(sess, form_data)
    cards = json_data["data"]

    # If there's more than PAGE_MAX results, keep calling the scrape url until we have all of the card records
    total_cards = json_data["recordsTotal"]
    if total_cards > PAGE_MAX:
        additional_pages = math.ceil((total_cards - PAGE_MAX) / PAGE_MAX)
        for i in range(additional_pages):
            curr_page = i + 1
            form_data = {
                "headingID": SET_ID,
                "categoryID": "20019",
                "draw": curr_page + 2,
                "start": PAGE_MAX * curr_page,
                "length": PAGE_MAX,
                "isPSADNA": "false"
            }

            json_data = post_to_url(sess, form_data)
            cards += json_data["data"]

    # Create a dataframe
    df = pd.DataFrame(cards[1:]).drop_duplicates()
    save_df(df, set_name)

# Output the dataframe as a file
def save_df(df, set_name):
    df.to_csv(get_file_name(set_name), index = False)
    date = "09_09_2023"
    drive.mount('/content/gdrive')
    destination = '/content/gdrive/My Drive/BaseballCardsInfo/' + get_file_name(set_name)
    print(destination)
    df.to_csv(destination, index = False)

def post_to_url(session, form_data):
    r = session.post(POP_URL_BASE, data=form_data)
    r.raise_for_status()
    json_data = r.json()
    time.sleep(3)
    return json_data

def get_file_name(set_name):
    f_name = "{}--{}".format(set_name.replace(" ", "-"), str(time.strftime("%Y-%m-%d-%H%M%S")))
    return "{}.csv".format(f_name)

url = "https://www.psacard.com/pop/baseball-cards/1952/topps/49722"
set_name = "1952 Topps"

# Initialize class and execute web scraping
ppr = scrape(url, set_name)

Mounted at /content/gdrive
/content/gdrive/My Drive/BaseballCardsInfo/1952-Topps--2024-03-13-012742.csv
